# Supplementary Tables
- goal: Generate all Supplementary Tables in long format (easier for reader)
- input: results
- output: Supplementary Tables as excel files with multiple sheets

requires `xlsxwriter`
- `conda install -c conda-forge xlsxwriter` did not work
- `pip install XlsxWriter` works

In [1]:
cd ../

/home/sreichl/projects/bmdm-stim


In [2]:
# libraries
import pandas as pd
import os
import re
from functools import reduce

In [23]:
# utility functions

# Function to apply replacements to a single index value
def apply_replacements(index_value):
    if type(index_value)==str:
        return re.sub('|'.join(re.escape(key) for key in replacements.keys()), 
                      lambda m: replacements[m.group(0)], index_value)
    else:
        return index_value
    
# Function to apply replacements to a single index value
def apply_bulk_sample_name_replacements(index_value):
    if type(index_value)==str:
        return re.sub('|'.join(re.escape(key) for key in bulk_sample_name_replacements.keys()), 
                      lambda m: bulk_sample_name_replacements[m.group(0)], index_value)
    else:
        return index_value

def df_to_excel(writer, df, sheet_name, column_width, index_format):
    
    ### replace terms using pre defined dictionary
    # apply replacements to dataframe content
    string_columns = df.select_dtypes(include=['object']).columns
    df[string_columns] = df[string_columns].replace(replacements, regex=True)
    
    ## column names
    # apply replacements to column names
    df.columns = [re.sub('|'.join(re.escape(key) for key in replacements.keys()), 
                         lambda m: replacements[m.group(0)], col) for col in df.columns]
    # Replace underscores with spaces and capitalize the first letter of each column name
    df.columns = [col.replace('_', ' ')[0].upper() + col.replace('_', ' ')[1:] for col in df.columns]
    
    ## indices
    # apply replacements to each (Multi)Index content
    if isinstance(df.index, pd.MultiIndex):
        df.index = df.index.map(lambda x: tuple(apply_replacements(level) for level in x))
    else:
        df.index = df.index.map(apply_replacements)
    # apply replacements to index names
    df.index.names = [re.sub('|'.join(re.escape(key) for key in replacements.keys()), 
                         lambda m: replacements[m.group(0)], col) for col in df.index.names]
    # Capitalize only the first letter of each index level name, preserving the rest
    df.index.names = [name[0].upper() + name[1:] if name is not None else None for name in df.index.names]
    
    ### create Excel sheet
    idx_nums = len(df.index.names)
    
    # Write table as a separate sheet to the Excel file
    df.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=1, startcol=idx_nums, freeze_panes = (1,idx_nums))
    
    # set all columns to same width
    writer.sheets[sheet_name].set_column(first_col=0, last_col=df.shape[1]+idx_nums-1, width = column_width)
    
    # Write the column headers with the defined format.
    for col_num, value in enumerate(df.columns.values):
        writer.sheets[sheet_name].write(0, col_num + idx_nums, value, index_format)

    # Write the index names with the defined format
    for idx_name_num, value in enumerate(df.index.names):
        # write the index name/header
        writer.sheets[sheet_name].write(0, idx_name_num, value, index_format)
        
        # Write the index values with the defined format for each index.
        for idx_num, idx_value in enumerate(df.index.values):
            if idx_nums == 1:
                writer.sheets[sheet_name].write(idx_num + 1, idx_name_num, idx_value, index_format)
            else:
                writer.sheets[sheet_name].write(idx_num + 1, idx_name_num, idx_value[idx_name_num], index_format)

    return writer

In [28]:
def rearrange_sample_names(sample_name):
#     RNA_{stimulus}_{time}_{replicate}_{batch}
#     ATAC_{stimulus}_{time}_{replicate}_{batch}
    parts = sample_name.split('_')
    rearranged_parts = [parts[0], parts[3], parts[4], parts[2], parts[1]]
    return '_'.join(rearranged_parts)

In [37]:
# configs
results_path = os.path.join('results', 'tables')
column_width = 25

# define replacement dictionary
replacements = {
    'PT76': '1',
    'PT82': '2',
    'C_albicans': 'Candida',
    'LCMV_Cl13': 'LCMV',
    'experiment_id': 'batch',
    'Conditions': 'Time points',
    'Condition': 'Time point',
    'Treatment_time': 'Time point',
    'Treatment': 'Stimulus',
    'group': 'Comparison',
#      'logFC': 'Coefficient', or 'Log fold change'?
    'AveExpr': 'Average expression',
    'P.Value': 'P-value',
    'adj.P.Val': 'Adjusted P-value',
    'untreated': 'Untreated',
    'LO28_2h': 'Listeria 2h',
    'LO28_6h': 'Listeria 6h',
    'LO28-6h': 'Listeria 6h',
    'LO28-24h': 'Listeria 24h',
    'LO28': 'Listeria',
    'p_val_adj': 'Adjusted P-value',
    'p_val': 'P-value',
    #      'avg_log2FC': 'Coefficient', or 'Average log2 fold change'?
    'pct.': 'Percentage expression ',
#     "_2h": " 2h",
#     "_4h": " 4h",
#     "_6h": " 6h",
#     "_8h": " 8h",
#     "_24h": " 24h",
    "mSpi1": "Spi1",
    "mNon-targeting": "WT",
}


bulk_sample_name_replacements = {
    'PT76': '1',
    'PT82': '2',
    'C_albicans': 'Candida',
    'LCMV_Cl13': 'LCMV',
    'IFN_beta': 'IFNbeta',
    'IFN_gamma': 'IFNgamma',
    'LO28': 'Listeria',
}

In [5]:
# config the xlsxwriter format for sheets
xlsx_format_dict = {'bold': True,
   'font_color': 'black',
#    'locked': True,
#    'text_wrap': True,
#    'bg_color': '#FDFD96',
   'border': False,
  'align': 'left',
# 'valign': 'vcenter'
  }

In [6]:
if not os.path.exists(results_path):
        os.mkdir(results_path)

In [7]:
# load bulk time series configs
timeseries_k = pd.read_csv(os.path.join('config','BMDM_timeseries_k.csv'), index_col=0)
# timeseries_enrichment_terms = pd.read_csv(os.path.join('config','BMDM_timeseries_enrichment_terms.csv'), index_col=0) # NOT USED in main figures
timeseries_k

,RNA,ATAC,INT
LO28,10,10,7
LPS,8,6,6
LCMV_Cl13,2,3,4
C_albicans,2,5,6
IFN_beta,4,8,4
IFN_gamma,3,6,5


In [8]:
# load small CROP-seq configs
spi1_enrichment_terms = pd.read_csv(os.path.join('config','Spi1KO_enrichment_terms.csv'), index_col=0)
spi1_enrichment_terms

,database,terms
Spi1_plot,BioPlanet_2019,Oncostatin M | Interferon alpha/beta signaling...


In [9]:
# load gene annotations
gene_annot = pd.read_csv(os.path.join('results', 'RNA', 'counts', 'gene_annotation.tsv'), index_col=0, sep = '\t')
# Rename column from 'external_gene_name' to 'symbol'
gene_annot.rename(columns={'external_gene_name': 'symbol'}, inplace=True)
print(gene_annot.shape)
gene_annot.head()

(55487, 8)


,version,source,symbol,external_gene_source,description,gene_biotype,length,gc
ensembl_gene_id,,,,,,,,
ENSMUSG00000000194,13,ensembl_havana,Gpr107,MGI Symbol,G protein-coupled receptor 107 [Source:MGI Sym...,protein_coding,12766,0.494595
ENSMUSG00000000247,11,ensembl_havana,Lhx2,MGI Symbol,LIM homeobox protein 2 [Source:MGI Symbol;Acc:...,protein_coding,4393,0.597086
ENSMUSG00000000544,14,ensembl_havana,Gpa33,MGI Symbol,glycoprotein A33 (transmembrane) [Source:MGI S...,protein_coding,2742,0.530999
ENSMUSG00000000817,10,ensembl_havana,Fasl,MGI Symbol,"Fas ligand (TNF superfamily, member 6) [Source...",protein_coding,1937,0.430563
ENSMUSG00000000889,8,ensembl_havana,Dbh,MGI Symbol,dopamine beta hydroxylase [Source:MGI Symbol;A...,protein_coding,2692,0.554978


# RNA - TableS1

In [38]:
data_path = os.path.join('results', 'RNA')

In [39]:
# Create Excel writer
writer = pd.ExcelWriter(os.path.join(results_path,'S01_RNAseq.xlsx'), engine='xlsxwriter')

# Get the workbook and worksheet objects
workbook = writer.book
# define specific formatting
index_format = workbook.add_format(xlsx_format_dict)

In [40]:
# add an empty sheet called LEGEND (fill with manually curated data later)
pd.DataFrame().to_excel(writer, sheet_name="TableS1_legends", index=False)

In [41]:
# metadata
metadata = pd.read_csv(os.path.join('metadata', 'RNA_sample_metadata.csv'), index_col=0)
# load statistics
statistics = pd.read_csv(os.path.join('metadata', 'RNA_multiqc_report.csv'), index_col=0)
# concatenate dataframes
metadata = pd.concat([metadata, statistics], axis=1)
# remove 2 samples by QC
metadata.drop(['RNA_PT82_R1_untreated_8h','RNA_PT82_R1_LPS_24h'], axis=0, inplace=True)
# remove unused columns
metadata.drop(['mouse_pool'], axis=1, inplace=True)
# remove experiment id from sample name
# metadata.index = metadata.index.astype(str).str.replace('PT\d{2}_', '', regex=True)
# add index name
metadata.index.names = ["sample"]

metadata_copy = metadata.copy(deep=True)
metadata_copy.index = metadata_copy.index.map(apply_bulk_sample_name_replacements)
metadata_copy.index = [rearrange_sample_names(col) for col in metadata_copy.index]
metadata_copy.index.names = ["sample"]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = metadata_copy,
                     sheet_name = "sample_annotation",
                     column_width = 25,
                     index_format = index_format
                    )

print(metadata.shape)
metadata.head()

(64, 11)


,library,timepoint after thawing,cell_line,Treatment,Treatment_time,sex,organism,bio-replicate,experiment_id,Aligned_percentage,Aligned_million
sample,,,,,,,,,,,
RNA_PT76_R1_C_albicans_2h,Quant-seq,d010,BMDM,C_albicans,2h,female,mouse,R1,PT76,0.740,4.6
RNA_PT82_R1_C_albicans_24h,Quant-seq,d010,BMDM,C_albicans,24h,female,mouse,R1,PT82,0.739,4.3
RNA_PT76_R1_C_albicans_4h,Quant-seq,d010,BMDM,C_albicans,4h,female,mouse,R1,PT76,0.745,3.5
RNA_PT76_R1_C_albicans_6h,Quant-seq,d010,BMDM,C_albicans,6h,female,mouse,R1,PT76,0.741,4.3
RNA_PT82_R1_C_albicans_8h,Quant-seq,d010,BMDM,C_albicans,8h,female,mouse,R1,PT82,0.722,3.9


In [42]:
# DEA results
dea_dfs = []

for treatment in metadata["Treatment"].unique():
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'DEA', 'DEA_{}.tsv'.format(treatment)), delimiter="\t")

    # Drop columns 'B' and 't'
    df_tmp = df_tmp.drop(['B', 't'], axis=1)

    # Rename column from 'rn' to 'gene'
    df_tmp.rename(columns={'rn': 'gene'}, inplace=True)
    
    # Add gene symbols using gene annotation
    df_tmp = pd.merge(df_tmp, gene_annot[['symbol']], left_on='gene', right_index=True, how='left')
    
    # set indices
    df_tmp.set_index(["gene", "symbol", "group"], inplace=True)

    # Pivot the data
#     df_tmp = df_tmp.pivot(index='gene', columns='group')

    # Flatten the multi-index columns and reverse the order of concatenation
#     df_tmp.columns = ['_'.join(col[::-1]).strip() for col in df_pivot.columns.values]
    
    dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)

# filter for genes that are at least in one comparison fulfill the most permissive criteria (C.albicans)
genes_sig = dea_df.index[(dea_df["adj.P.Val"]<0.05) & (abs(dea_df["logFC"])>1) & (dea_df["AveExpr"]>1)].get_level_values('gene').unique().tolist()
dea_df = dea_df.loc[genes_sig,:]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "differential_expression",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(263970, 4)


LogFC  Average expression  \
Gene               Symbol Comparison                                  
ENSMUSG00000038437 Mllt6  Candida_4h   1.583815            5.750605   
                          Candida_2h   1.709446            5.750605   
                          Candida_24h  0.173672            5.750605   
                          Candida_8h   0.738289            5.750605   
                          Candida_6h   1.236302            5.750605   

                                            P-value  Adjusted P-value  
Gene               Symbol Comparison                                   
ENSMUSG00000038437 Mllt6  Candida_4h   1.885853e-10      2.400503e-06  
                          Candida_2h   2.153350e-11      3.915714e-08  
                          Candida_24h  3.405508e-01      9.999846e-01  
                          Candida_8h   1.223462e-04      3.893363e-02  
                          Candida_6h   6.920095e-08      8.808589e-05

In [43]:
# Enrichment results of DEG clusters (term X cluster_statistic)
enr_dfs = []

for cluster in range(1,9):
    # load enr results
    df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'DEG_clusters', 'k_8', 'cluster_{}'.format(cluster), 'GO_Biological_Process_2021', 'Enrichr_GO_Biological_Process_2021_cluster_{}.csv'.format(cluster)))

    # Drop columns
    df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)
#     df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set', 'Genes'], axis=1)

    # Rename columns 
#     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

    # add cluster column
    df_tmp["Cluster"] = cluster
    
    # set indices
    df_tmp.set_index(["Term", "Cluster"], inplace=True)
    
    enr_dfs.append(df_tmp)

enr_df = pd.concat(enr_dfs, axis=0)
# enr_df = pd.concat(enr_dfs, axis=1)

# add database column
enr_df["Database"] = 'GO Biological Process 2021'

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = enr_df,
                     sheet_name = "diff_expression_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(enr_df.shape)
enr_df.head()

(15418, 6)


,,Overlap,P-value,Adjusted P-value,Genes,Odds Ratio,Database
Term,Cluster,,,,,,
'de novo' posttranslational protein folding (GO:0051084),1,1/28,0.987242,1.000000,DNAJB5,0.247448,GO Biological Process 2021
2-oxoglutarate metabolic process (GO:0006103),1,3/11,0.203409,0.867825,IDH1;IDH2;D2HGDH,1.894196,GO Biological Process 2021
3'-UTR-mediated mRNA destabilization (GO:0061158),1,1/13,0.867838,1.000000,ZFP36L1,0.533594,GO Biological Process 2021
3'-UTR-mediated mRNA stabilization (GO:0070935),1,1/11,0.819539,1.000000,MAPK14,0.630710,GO Biological Process 2021
3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427),1,2/11,0.485046,0.978915,ENPP1;ABHD14B,1.262108,GO Biological Process 2021


In [44]:
# Time series clustering (gene X stimulus matrix with cluster membership or empty)
ts_dfs = []

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
    
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'time_series', treatment, 'k_{}'.format(timeseries_k.loc[treatment,"RNA"]), 'clustering_{}.csv'.format(treatment)), index_col=[1,2])
    
    # Drop columns
    df_tmp = df_tmp.drop(['Unnamed: 0'], axis=1)

    # Rename columns 
    df_tmp.columns = ["{}".format(treatment)]
    
    ts_dfs.append(df_tmp)
    
ts_df = pd.concat(ts_dfs, axis=1)
ts_df.index.names = ["gene", "symbol"]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = ts_df,
                     sheet_name = "time_series_clusters",
                     column_width = 25,
                     index_format = index_format
                    )

print(ts_df.shape)
ts_df.head()

(4775, 6)


,,Candida,IFN beta,IFN gamma,LCMV,Listeria,LPS
Gene,Symbol,,,,,,
ENSMUSG00000000028,Cdc45,NaN,1.0,NaN,NaN,8.0,4.0
ENSMUSG00000000056,Narf,NaN,4.0,NaN,NaN,1.0,7.0
ENSMUSG00000000058,Cav2,NaN,4.0,NaN,NaN,NaN,7.0
ENSMUSG00000000085,Scmh1,NaN,1.0,NaN,NaN,7.0,1.0
ENSMUSG00000000148,Brat1,NaN,4.0,NaN,NaN,1.0,NaN


In [45]:
# Enrichment results of time-series clusters (term X cluster_statistic)
ts_enr_dfs = []

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
        
    k_tmp = timeseries_k.loc[treatment,"RNA"]

    for cluster in range(1,k_tmp+1):
        # load enr results
        df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'time_series', treatment, 'k_{}'.format(k_tmp), 'cluster_{}'.format(cluster), 'BioPlanet_2019', 'Enrichr_BioPlanet_2019_cluster_{}.csv'.format(cluster)))

        # Drop columns
        df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)
    #     df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set', 'Genes'], axis=1)

        # Rename columns 
    #     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

        # add cluster column
        df_tmp["Treatment"] = treatment
        df_tmp["Cluster"] = cluster
        
        # set indices
        df_tmp.set_index(["Term", "Treatment", "Cluster"], inplace=True)

        ts_enr_dfs.append(df_tmp)

ts_enr_df = pd.concat(ts_enr_dfs, axis=0)
# ts_enr_df = pd.concat(ts_enr_dfs, axis=1)

# add database column
ts_enr_df["Database"] = 'BioPlanet 2019'

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = ts_enr_df,
                     sheet_name = "time_series_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(ts_enr_df.shape)
ts_enr_df.head()

(17401, 6)


,,,Overlap,P-value,Adjusted P-value,Genes,Odds Ratio,Database
Term,Stimulus,Cluster,,,,,,
4-1BB-dependent immune response,Candida,1,2/17,0.017993,0.057914,NFKB1;NFKBIA,9.710466,BioPlanet 2019
ABC transporters,Candida,1,1/26,0.274543,0.360688,ABCC5,3.151861,BioPlanet 2019
AKT signaling pathway,Candida,1,2/19,0.022265,0.065900,NFKB1;NFKBIA,8.686945,BioPlanet 2019
AP-1 transcription factor network,Candida,1,2/52,0.133319,0.233126,BCL2L11;FOS,3.165834,BioPlanet 2019
ATF2 transcription factor network,Candida,1,2/44,0.101181,0.199649,DUSP5;FOS,3.743802,BioPlanet 2019


In [46]:
# Save Excel file
writer.save()

# ATAC - TableS2
- Homer_gene annotations in DEA and ts clustering results are not indices as soemtimes NA/Inf related error in XLSX writer

In [47]:
data_path = os.path.join('results', 'ATAC', 'all')

In [48]:
# Create Excel writer
writer = pd.ExcelWriter(os.path.join(results_path,'S02_ATACseq.xlsx'), engine='xlsxwriter')

# Get the workbook and worksheet objects
workbook = writer.book
# define specific formatting
index_format = workbook.add_format(xlsx_format_dict)

In [49]:
# add an empty sheet called LEGEND (fill with manually curated data later)
pd.DataFrame().to_excel(writer, sheet_name="TableS2_legends", index=False)

In [55]:
# metadata
metadata = pd.read_csv(os.path.join('metadata', 'ATAC_sample_metadata.csv'), index_col=0)
# load statistics
statistics = pd.read_csv(os.path.join('metadata', 'ATAC_multiqc_report.csv'), index_col=0)
# concatenate dataframes
metadata = pd.concat([metadata, statistics], axis=1)
# remove unused columns
metadata.drop(['mouse_pool','flowcell'], axis=1, inplace=True)
# remove experiment id from sample name
# metadata.index = metadata.index.astype(str).str.replace('PT\d{2}', 'ATAC', regex=True)
# add index name
metadata.index.names = ["sample"]

metadata_copy = metadata.copy(deep=True)
metadata_copy.index = "ATAC_"+metadata.index
metadata_copy.index = metadata_copy.index.map(apply_bulk_sample_name_replacements)
metadata_copy.index = [rearrange_sample_names(col) for col in metadata_copy.index]
metadata_copy.index.names = ["sample"]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = metadata_copy,
                     sheet_name = "sample_annotation",
                     column_width = 25,
                     index_format = index_format
                    )

print(metadata.shape)
metadata.head()

(78, 26)


,library,timepoint after thawing,cell_line,Treatment,Treatment_time,sex,bio-replicate,organism,experiment_id,M Reads,...,Fragment Length,Treatment Redundancy,% Dups,% Aligned,% Duplication,% > Q30,Mb Q30 bases,GC content,% PF,% Adapter
sample,,,,,,,,,,,,,,,,,,,,,
PT76_R1_C_albicans_2h,ATAC-seq,d010,BMDM,C_albicans,2h,female,R1,mouse,PT76,12.7,...,147.0,0.06,0.19,0.99,0.14,0.97,609.5,0.51,1.0,0.10
PT76_R1_C_albicans_4h,ATAC-seq,d010,BMDM,C_albicans,4h,female,R1,mouse,PT76,15.5,...,147.0,0.06,0.19,0.99,0.15,0.97,748.6,0.50,1.0,0.12
PT76_R1_C_albicans_6h,ATAC-seq,d010,BMDM,C_albicans,6h,female,R1,mouse,PT76,14.0,...,147.0,0.08,0.22,0.99,0.17,0.96,675.5,0.51,1.0,0.10
PT76_R1_C_albicans_8h,ATAC-seq,d010,BMDM,C_albicans,8h,female,R1,mouse,PT76,15.2,...,147.0,0.06,0.19,0.99,0.14,0.97,733.7,0.50,1.0,0.11
PT76_R1_untreated_0h,ATAC-seq,d010,BMDM,untreated,0h,female,R1,mouse,PT76,16.0,...,147.0,0.06,0.18,0.99,0.14,0.97,769.1,0.50,1.0,0.13


In [56]:
# consensus region annotation
region_annot = pd.read_csv(os.path.join(data_path, 'all_consensus_regions_annotation_filtered.csv'))

# Rename column from 'rn' to 'gene'
region_annot.rename(columns={'Unnamed: 0': 'region'}, inplace=True)

# set index
region_annot.set_index(["region"], inplace=True)

# filter & rename columns
region_annot = region_annot.loc[:,['gencode_chr','gencode_start','gencode_end', 'homer_Gene Name']]
region_annot.columns = ['Chromosome','Start','End','Homer_gene']

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = region_annot.copy(deep=True),
                     sheet_name = "consensus_regions",
                     column_width = 25,
                     index_format = index_format
                    )

print(region_annot.shape)
region_annot.head()

(136735, 4)


,Chromosome,Start,End,Homer_gene
region,,,,
CONS00000000005,chr1,4414052,4415256,Rp1
CONS00000000006,chr1,4416568,4417221,Rp1
CONS00000000011,chr1,4491871,4492835,Sox17
CONS00000000013,chr1,4622075,4622934,Sox17
CONS00000000017,chr1,4755824,4756551,Mrpl15


In [57]:
# DEA results
dea_dfs = []

for treatment in metadata["Treatment"].unique():
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'DEA', 'DEA_{}.tsv'.format(treatment)), delimiter="\t")

    # Drop columns 'B' and 't'
    df_tmp = df_tmp.drop(['B', 't'], axis=1)

    # Rename column from 'rn' to 'gene'
    df_tmp.rename(columns={'rn': 'region'}, inplace=True)
    
     # Add gene symbols using region annotation
    df_tmp = pd.merge(df_tmp, region_annot[['Homer_gene']], left_on='region', right_index=True, how='left')

    # Pivot the data
#     df_tmp = df_tmp.pivot(index='gene', columns='group')

    # Flatten the multi-index columns and reverse the order of concatenation
#     df_tmp.columns = ['_'.join(col[::-1]).strip() for col in df_pivot.columns.values]

    # set indices
    df_tmp.set_index(["region", "group"], inplace=True)
    
    # filter regions
    if treatment!='C_albicans':
        regions_sig = df_tmp.index[(df_tmp["adj.P.Val"]<0.05) & (abs(df_tmp["logFC"])>2) & (df_tmp["AveExpr"]>0)].get_level_values('region').unique().tolist()
    else:
        regions_sig = df_tmp.index[(df_tmp["adj.P.Val"]<0.05) & (abs(df_tmp["logFC"])>1) & (df_tmp["AveExpr"]>0)].get_level_values('region').unique().tolist()
    
    df_tmp = df_tmp.loc[regions_sig,:]
    
    dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)

# # filter for regions that at least in one comparison fulfill the standard criteria -> switched to treatment-wise filtering to reduce size
# regions_sig = dea_df.index[(dea_df["adj.P.Val"]<0.05) & (abs(dea_df["logFC"])>2) & (dea_df["AveExpr"]>0)].get_level_values('region').unique().tolist()
# # find C albicans specific regions with more leniant filtering
# Calbicans_regions_sig = dea_df.index[(dea_df["adj.P.Val"]<0.05) & (abs(dea_df["logFC"])>1) & (dea_df["AveExpr"]>0) & (dea_df.index.get_level_values('group').str.contains('C_albicans'))].get_level_values('region').unique().tolist()
# # make union of final relevant regions
# regions_sig = list(set(regions_sig).union(Calbicans_regions_sig))
# dea_df = dea_df.loc[regions_sig,:]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "differential_accessibility",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(176075, 5)


LogFC  Average expression       P-value  \
Region          Comparison                                                
CONS00000130796 Candida_2h   1.173784            4.369844  2.008776e-15   
                Candida_4h   0.696071            4.369844  4.387840e-07   
                Candida_6h   0.640373            4.369844  4.282775e-06   
                Candida_8h   0.592594            4.369844  2.730619e-08   
                Candida_24h  0.283090            4.369844  1.370579e-02   

                             Adjusted P-value Homer gene  
Region          Comparison                                
CONS00000130796 Candida_2h       2.746700e-10      Prdx1  
                Candida_4h       3.203916e-04      Prdx1  
                Candida_6h       1.796335e-03      Prdx1  
                Candida_8h       1.598495e-05      Prdx1  
                Candida_24h      7.027565e-01      Prdx1

In [58]:
# Enrichment results of DEG clusters (term X cluster_statistic)
enr_dfs = []

for cluster in range(1,10):
    # load enr results
    df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'DAR_clusters', 'k9_cluster_{}'.format(cluster), 'GREAT', 'GREAT_GO Biological Process.tsv'), sep="\t")

    # Rename columns 
#     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

    # add cluster column
    df_tmp["Cluster"] = cluster
    
    # set Term = name+ID
    df_tmp["Term"] = df_tmp["name"]+' ('+df_tmp["ID"]+')'
    
    # Drop columns
#     df_tmp = df_tmp.drop(['name', 'ID'], axis=1)
    # keep only selecteed columns
    df_tmp = df_tmp.loc[:,['Term','Cluster','Hyper_Fold_Enrichment','Hyper_Raw_PValue','Hyper_Adjp_BH']]
    
    # set indices
    df_tmp.set_index(["Term", "Cluster"], inplace=True)
    
    enr_dfs.append(df_tmp)

enr_df = pd.concat(enr_dfs, axis=0)
# enr_df = pd.concat(enr_dfs, axis=1)

# add database column
enr_df["Database"] = 'GO Biological Process'

# filter enrichment results by significance ie keep only terms that have at least one significant value
significant_terms = enr_df.groupby(level='Term').apply(lambda group: (group['Hyper_Adjp_BH'] < 0.05).any())
significant_terms = significant_terms[significant_terms]  
enr_df = enr_df.loc[enr_df.index.get_level_values('Term').isin(significant_terms.index)]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = enr_df,
                     sheet_name = "diff_accessibility_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(enr_df.shape)
enr_df.head()

(12222, 4)


,,Hyper Fold Enrichment,Hyper Raw PValue,Hyper Adjp BH,Database
Term,Cluster,,,,
regulation of innate immune response (GO:0045088),1,1.947448,1.769698e-14,2.316535e-10,GO Biological Process
immune response (GO:0006955),1,1.419579,1.906900e-11,1.248066e-07,GO Biological Process
regulation of response to stress (GO:0080134),1,1.310585,8.841341e-11,3.857772e-07,GO Biological Process
regulation of defense response (GO:0031347),1,1.482990,4.155906e-10,1.322796e-06,GO Biological Process
innate immune response (GO:0045087),1,1.581043,5.202335e-10,1.322796e-06,GO Biological Process


In [59]:
# Time series clustering (gene X stimulus matrix with cluster membership or empty)
ts_dfs = []

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
    
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'time_series', treatment, 'k_{}'.format(timeseries_k.loc[treatment,"ATAC"]), 'clustering_{}.csv'.format(treatment)), index_col=1)
    
    # Drop columns
    df_tmp = df_tmp.drop(['Unnamed: 0'], axis=1)

    # Rename columns 
    df_tmp.columns = ["{}".format(treatment)]
    
    ts_dfs.append(df_tmp)
    
ts_df = pd.concat(ts_dfs, axis=1)
ts_df.index.names = ["region"]

# Add gene symbols using region annotation
ts_df = pd.merge(ts_df, region_annot[['Homer_gene']], left_on='region', right_index=True, how='left')

# set indices
# ts_df.reset_index(inplace=True)
# ts_df.set_index(["region","Homer_gene"], inplace=True)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = ts_df,
                     sheet_name = "time_series_clusters",
                     column_width = 25,
                     index_format = index_format
                    )

print(ts_df.shape)
ts_df.head()

(19238, 7)


,Candida,IFN beta,IFN gamma,LCMV,Listeria,LPS,Homer gene
Region,,,,,,,
CONS00000130796,2.0,NaN,NaN,NaN,NaN,NaN,Prdx1
CONS00000032052,2.0,4.0,NaN,NaN,6.0,4.0,Ccl5
CONS00000021736,2.0,NaN,NaN,NaN,NaN,NaN,Tmcc3
CONS00000173048,2.0,5.0,NaN,NaN,NaN,NaN,Lpl
CONS00000017578,2.0,NaN,NaN,NaN,NaN,NaN,Prdm1


In [60]:
# Enrichment results of time-series clusters (term X cluster_statistic)
ts_enr_dfs = []

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
        
    k_tmp = timeseries_k.loc[treatment,"ATAC"]

    for cluster in range(1,k_tmp+1):
        # load enr results
        df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'time_series', '{}_k{}_cluster_{}'.format(treatment, k_tmp, cluster), 'GREAT', 'GREAT_GO Biological Process.tsv'), sep="\t")

        # Rename columns 
    #     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

        # add cluster column
        df_tmp["Treatment"] = treatment
        df_tmp["Cluster"] = cluster
        
        # set Term = name+ID
        df_tmp["Term"] = df_tmp["name"]+' ('+df_tmp["ID"]+')'
    
        # Drop columns
#         df_tmp = df_tmp.drop(['name', 'ID'], axis=1)
        # keep only selecteed columns
        df_tmp = df_tmp.loc[:,['Term','Treatment','Cluster','Hyper_Fold_Enrichment','Hyper_Raw_PValue','Hyper_Adjp_BH']]
    
        # set indices
        df_tmp.set_index(["Term", "Treatment", "Cluster"], inplace=True)

        ts_enr_dfs.append(df_tmp)

ts_enr_df = pd.concat(ts_enr_dfs, axis=0)
# ts_enr_df = pd.concat(ts_enr_dfs, axis=1)

# add database column
ts_enr_df["Database"] = 'GO Biological Process'

# filter enrichment results by significance ie keep only terms that have at least one significant value
significant_terms = ts_enr_df.groupby(level='Term').apply(lambda group: (group['Hyper_Adjp_BH'] < 0.05).any())
significant_terms = significant_terms[significant_terms]  
ts_enr_df = ts_enr_df.loc[ts_enr_df.index.get_level_values('Term').isin(significant_terms.index)]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = ts_enr_df,
                     sheet_name = "time_series_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(ts_enr_df.shape)
ts_enr_df.head()

(59888, 4)


,,,Hyper Fold Enrichment,Hyper Raw PValue,Hyper Adjp BH,Database
Term,Stimulus,Cluster,,,,
regulation of substrate adhesion-dependent cell spreading (GO:1900024),Candida,1,4.858430,7.389429e-07,0.009673,GO Biological Process
negative regulation of amyloid precursor protein biosynthetic process (GO:0042985),Candida,1,13.947210,2.112510e-04,0.420393,GO Biological Process
macromolecule catabolic process (GO:0009057),Candida,1,1.727161,3.556155e-04,0.479560,GO Biological Process
positive regulation of cell adhesion (GO:0045785),Candida,1,1.765998,8.793954e-04,0.639516,GO Biological Process
cellular macromolecule catabolic process (GO:0044265),Candida,1,1.707647,9.779085e-04,0.673727,GO Biological Process


In [61]:
# Save Excel file
writer.save()

# INT - TableS3

In [29]:
data_path = os.path.join('results', 'INT')

In [30]:
# Create Excel writer
writer = pd.ExcelWriter(os.path.join(results_path,'S03_RNAseq_ATACseq_integrative_analysis.xlsx'), engine='xlsxwriter')

# Get the workbook and worksheet objects
workbook = writer.book
# define specific formatting
index_format = workbook.add_format(xlsx_format_dict)

In [31]:
# add an empty sheet called LEGEND (fill with manually curated data later)
pd.DataFrame().to_excel(writer, sheet_name="TableS3_legends", index=False)

In [32]:
# DEA results

dea_df = pd.read_csv(os.path.join(data_path, 'DEA', 'INT_DEA_all.csv'))

# Rename column from 'rn' to 'gene'
dea_df.rename(columns={'rn': 'gene'}, inplace=True)

# Add gene symbols using gene annotation
dea_df = pd.merge(dea_df, gene_annot[['symbol']], left_on='gene', right_index=True, how='left')

# set indices
dea_df.set_index(["gene","symbol", "group"], inplace=True)

# Drop columns
dea_df = dea_df.drop(['time', 'treatment'], axis=1)
dea_df = dea_df.drop(['B', 't'], axis=1)

# filter for genes that are at least in one comparison fulfill the criteria
genes_sig = dea_df.index[(dea_df["adj.P.Val"]<0.05) & (abs(dea_df["logFC"])>1) & (dea_df["AveExpr"]>1)].get_level_values('gene').unique().tolist()
dea_df = dea_df.loc[genes_sig,:]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "divergent_gene_identification",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(225122, 4)


LogFC  Average expression  \
Gene               Symbol Comparison                                  
ENSMUSG00000074141 Il4i1  Candida 24h  3.248002            3.743529   
                          Candida 2h   0.039455            3.743529   
                          Candida 4h   0.171906            3.743529   
                          Candida 6h   0.307128            3.743529   
                          Candida 8h   1.448020            3.743529   

                                            P-value  Adjusted P-value  
Gene               Symbol Comparison                                   
ENSMUSG00000074141 Il4i1  Candida 24h  3.703750e-12      8.346216e-08  
                          Candida 2h   9.081105e-01      9.886195e-01  
                          Candida 4h   6.198277e-01      9.499374e-01  
                          Candida 6h   3.742488e-01      9.231326e-01  
                          Candida 8h   2.089307e-04      2.263534e-02

In [33]:
# Time series clustering (gene X stimulus matrix with cluster membership or empty)
ts_dfs = []

# Create a mapping from numbers to letters for clusters
num_to_letter = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G'}
listeria_changes = {
    'G': 'A',
    'A': 'B',
    'B': 'C',
    'D': 'D',
    'E': 'E',
    'C': 'F',
    'F': 'G',
}

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
    
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'time_series', treatment, 'k_{}'.format(timeseries_k.loc[treatment,"INT"]), 'clustering_{}.csv'.format(treatment)), index_col=[1,2])
    
    # Drop columns
    df_tmp = df_tmp.drop(['Unnamed: 0'], axis=1)

    # Rename columns 
    df_tmp.columns = ["{}".format(treatment)]
    
    ts_dfs.append(df_tmp)
    
ts_df = pd.concat(ts_dfs, axis=1)
ts_df.index.names = ["gene", "symbol"]

# Replace cluster numbers with letters
ts_df = ts_df.replace(num_to_letter)
# replace Listeria clusters accordingly
ts_df['LO28'] = ts_df['LO28'].replace(listeria_changes)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = ts_df,
                     sheet_name = "time_series_clusters",
                     column_width = 25,
                     index_format = index_format
                    )

print(ts_df.shape)
ts_df.head()

(7252, 6)


,,Candida,IFN beta,IFN gamma,LCMV,Listeria,LPS
Gene,Symbol,,,,,,
ENSMUSG00000000028,Cdc45,NaN,B,NaN,NaN,G,B
ENSMUSG00000000056,Narf,NaN,A,NaN,NaN,NaN,NaN
ENSMUSG00000000058,Cav2,NaN,NaN,NaN,NaN,NaN,F
ENSMUSG00000000127,Fer,NaN,NaN,NaN,NaN,D,NaN
ENSMUSG00000000149,Gna12,NaN,NaN,NaN,NaN,E,NaN


In [34]:
# Enrichment results of time-series clusters (term X cluster_statistic)
ts_enr_dfs = []

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
        
    k_tmp = timeseries_k.loc[treatment,"INT"]

    for cluster in range(1,k_tmp+1):
        # load enr results
        df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'time_series', treatment, 'k_{}'.format(k_tmp), 'cluster_{}'.format(cluster), 'GO_Biological_Process_2021', 'Enrichr_GO_Biological_Process_2021_cluster_{}.csv'.format(cluster)))

        # Drop columns
        df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)
    #     df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set', 'Genes'], axis=1)

        # Rename columns 
    #     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

        # add cluster column
        df_tmp["Treatment"] = treatment
        df_tmp["Cluster"] = cluster
        # rename clusters from numbers to letters
        df_tmp['Cluster'] = df_tmp['Cluster'].replace(num_to_letter)
        if treatment=="LO28":
            df_tmp['Cluster'] = df_tmp['Cluster'].replace(listeria_changes)
        
        # set indices
        df_tmp.set_index(["Term", "Treatment", "Cluster"], inplace=True)

        ts_enr_dfs.append(df_tmp)

ts_enr_df = pd.concat(ts_enr_dfs, axis=0)
# ts_enr_df = pd.concat(ts_enr_dfs, axis=1)

# add database column
ts_enr_df["Database"] = 'GO Biological Process 2021'

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = ts_enr_df,
                     sheet_name = "time_series_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(ts_enr_df.shape)
ts_enr_df.head()

(57202, 6)


,,,Overlap,P-value,Adjusted P-value,Odds Ratio,Genes,Database
Term,Stimulus,Cluster,,,,,,
3'-UTR-mediated mRNA destabilization (GO:0061158),Candida,A,1/16,0.032235,0.074873,30.880495,ZC3H12A,GO Biological Process 2021
B cell activation (GO:0042113),Candida,A,1/78,0.147724,0.178144,6.325726,CD40,GO Biological Process 2021
B cell proliferation (GO:0042100),Candida,A,1/26,0.051858,0.092375,18.999155,CD40,GO Biological Process 2021
B cell receptor signaling pathway (GO:0050853),Candida,A,1/32,0.063442,0.100254,15.434753,LCK,GO Biological Process 2021
CD40 signaling pathway (GO:0023035),Candida,A,1/9,0.018261,0.065709,54.907204,CD40,GO Biological Process 2021


In [35]:
# TFBS Enrichment results of time-series clusters (term X cluster_statistic)
tf_enr_dfs = []

cluster_to_letter = {"cluster_1": 'A', "cluster_2": 'B', "cluster_3": 'C', "cluster_4": 'D', "cluster_5": 'E', "cluster_6": 'F', "cluster_7": 'G'}

for treatment in metadata["Treatment"].unique():
    if treatment=="untreated":
        continue
        
    k_tmp = timeseries_k.loc[treatment,"INT"]

    # load TF enr results
    df_tmp = pd.read_csv(os.path.join(data_path, 'TF', 'INT_TF_time_series_{}_k{}.csv'.format(treatment, k_tmp)))

    # Drop columns
    df_tmp = df_tmp.drop(['AUC', 'TF_lowConf','rankAtMax'], axis=1)

    # Rename columns 
    df_tmp.rename(columns={"geneSet": "Cluster"}, inplace=True)
    
    # rename clusters from numbers to letters
    df_tmp['Cluster'] = df_tmp['Cluster'].replace(cluster_to_letter)
    if treatment=="LO28":
        df_tmp['Cluster'] = df_tmp['Cluster'].replace(listeria_changes)

    # add columns
    df_tmp["Treatment"] = treatment
        
    # set indices
    df_tmp.set_index(["Treatment", "Cluster"], inplace=True)

    tf_enr_dfs.append(df_tmp)

tf_enr_df = pd.concat(tf_enr_dfs, axis=0)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = tf_enr_df,
                     sheet_name = "TFBS_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(tf_enr_df.shape)
tf_enr_df.head()

(6229, 5)


Motif   NES                     TF highConf  \
Stimulus Cluster                                                               
Candida  A         predrem__nrMotif347  4.73                             NaN   
         A        transfac_pro__M04860  4.55   Rxra (inferredBy_Orthology).    
         A          predrem__nrMotif82  4.37                             NaN   
         A         neph__UW.Motif.0464  4.24                             NaN   
         A                 hdpi__ABCF2  4.00  Abcf2 (inferredBy_Orthology).    

                  NEnrGenes                                      EnrichedGenes  
Stimulus Cluster                                                                
Candida  A               15  Arhgef3;Cdc42ep2;Chd7;Fmnl2;Gadd45b;Hivep3;Kcn...  
         A               13  Abtb2;Arhgap28;Cd40;Ciita;Gadd45b;Gm12216;Kdm6...  
         A               11  Arhgap28;Fscn1;Hivep3;Lck;Lmo4;Lpar1;Mxd1;Prdm...  
         A               14  Abtb2;Arhgap28;Arhgef3;Gadd45b;Gm12216;Iqsec2;...  
         A               20  Abtb2;Arhgap28;Arhgef3;Cd40;Chd7;Ciita;Cish;Fc...

In [36]:
# Save Excel file
writer.save()

# CROP-seq small - TableS4

In [37]:
data_path = os.path.join('results', 'KO15')
conditions = ["untreated", "LO28_2h", "LO28_6h"]

In [38]:
# Create Excel writer
writer = pd.ExcelWriter(os.path.join(results_path,'S04_CROPseq_small.xlsx'), engine='xlsxwriter')

# Get the workbook and worksheet objects
workbook = writer.book
# define specific formatting
index_format = workbook.add_format(xlsx_format_dict)

In [39]:
# add an empty sheet called LEGEND (fill with manually curated data later)
pd.DataFrame().to_excel(writer, sheet_name="TableS4_legends", index=False)

In [40]:
# metadata
metadata = pd.read_csv(os.path.join(data_path, 'KO15_NORMALIZED_metadata.csv'), index_col=0)
# load mixscape classifications
mixscape_dfs = []
for cond in conditions:
    cond_tmp = pd.read_csv(os.path.join(data_path, 'mixscape', cond, 'KO15_mixscape_annotations.csv'), index_col=0)
    mixscape_dfs.append(cond_tmp)

# concatenate dfs
mixscape_df = pd.concat(mixscape_dfs, axis=0)
metadata = pd.concat([metadata, mixscape_df], axis=1)

# set condition column
metadata['Condition'] = metadata['hto_demux'].str.replace("HTO-RAW-SZ_", "")

# remove leading m in KO target gene names
metadata['KO_call'] = metadata['KO_call'].str[1:]
metadata['guide_call'] = metadata['guide_call'].str[1:]

# Drop columns
metadata = metadata.drop(['old.ident', 'NT_Spi1', "orig.ident", "pass_QC", "hto_demux", "nCount_HTO", "nFeature_HTO"], axis=1)

# add index name
metadata.index.names = ["Cell barcode"]

# set indices
metadata.reset_index(inplace=True)
metadata.set_index(["Cell barcode", "Condition", "KO_call"], inplace=True)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = metadata.copy(deep=True),
                     sheet_name = "single_cell_annotations",
                     column_width = 25,
                     index_format = index_format
                    )


# reset indices
metadata.reset_index(inplace=True)
metadata.set_index(["Cell barcode"], inplace=True)

print(metadata.shape)
metadata.head()

(9153, 19)


,Condition,KO_call,nCount_RNA,nFeature_RNA,nCount_AB,nFeature_AB,nCount_gRNA,nFeature_gRNA,batch,percent.mt,guide_call,nCount_SCT,nFeature_SCT,S.Score,G2M.Score,Phase,mixscape_class,mixscape_class_p_ko,mixscape_class.global
Cell barcode,,,,,,,,,,,,,,,,,,,
1_AAACCTGAGAAGAAGC-1,LO28_2h,Stat1,9229,2626,277,8,186,14,1,0.736808,Stat1_3,8846,2624,0.030562,-0.317354,S,Stat1 KO,0.86101,KO
1_AAACCTGAGACTGTAA-1,untreated,Irf8,10672,3572,389,10,1312,15,1,2.398801,Irf8_4,9274,3570,-0.093887,-0.181579,G1,Irf8 NP,0.00000,NP
1_AAACCTGAGGTGTTAA-1,untreated,Kdm1b,9867,3564,135,9,1069,11,1,1.530354,Kdm1b_3,9155,3561,-0.023333,0.346600,G2M,Kdm1b NP,0.00000,NP
1_AAACCTGCAGGCAGTA-1,LO28_6h,Non-targeting,8754,2577,232,9,43,16,1,1.028101,Non-targeting_6,8673,2575,-0.248403,-0.246516,G1,NT,0.00000,NT
1_AAACCTGCAGTCGTGC-1,LO28_6h,Hdac6,5200,1848,96,8,945,10,1,1.634615,Hdac6_2,7772,1868,-0.073559,-0.148819,G1,Hdac6 NP,0.00000,NP


In [41]:
# DEA - NT: 2h/6h vs UT results
dea_dfs = []

for cond in conditions:
    
    if cond=="untreated":
        continue
    
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'DEA_KO', 'KO15_DEA_RNA_mNon-targeting_{}_untreated.csv'.format(cond)), index_col=0)

    # set index name
    df_tmp.index.names = ["Gene"]
    
    # add column
    df_tmp["Time point"]=cond
    
    # set indices
    df_tmp.reset_index(inplace=True)
    df_tmp.set_index(["Gene", "Time point"], inplace=True)
    
    dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "DEA_in_WT",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(6663, 5)


,,P-value,Avg log2FC,Percentage expression 1,Percentage expression 2,Adjusted P-value
Gene,Time point,,,,,
Ccl4,Listeria 2h,6.375826e-92,4.746794,1.000,0.859,9.045384e-88
Tnf,Listeria 2h,1.676377e-90,4.563107,0.997,0.718,2.378276e-86
Junb,Listeria 2h,4.342102e-90,3.075322,1.000,0.725,6.160140e-86
Ccl3,Listeria 2h,1.005605e-89,3.470721,1.000,0.912,1.426651e-85
Cxcl2,Listeria 2h,3.237057e-83,5.308387,0.984,0.656,4.592413e-79


In [42]:
# DEA - UT/2h/6h: Spi1 vs NT results
dea_dfs = []

# KO15_DEA_RNA_LO28_2h_mSpi1.csv

for cond in conditions:
    
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'DEA_condition', 'KO15_DEA_RNA_{}_mSpi1.csv'.format(cond)), index_col=0)

    # set index name
    df_tmp.index.names = ["Gene"]
    
    # add column
    df_tmp["Time point"]=cond
    
    # set indices
    df_tmp.reset_index(inplace=True)
    df_tmp.set_index(["Gene", "Time point"], inplace=True)
    
    dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "DEA_Spi1",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(8086, 5)


,,P-value,Avg log2FC,Percentage expression 1,Percentage expression 2,Adjusted P-value
Gene,Time point,,,,,
Ptgds,Untreated,1.346231e-46,2.130750,0.730,0.050,1.909898e-42
Cst3,Untreated,2.992243e-28,-1.410878,0.992,1.000,4.245095e-24
Csf1r,Untreated,8.989721e-25,-1.197709,0.230,0.821,1.275372e-20
Tyrobp,Untreated,1.816720e-24,-0.955098,0.951,0.996,2.577380e-20
Coro1a,Untreated,2.867002e-24,-0.923344,0.910,0.996,4.067416e-20


In [43]:
# Enrichment results Spi1
spi1_enr_dfs = []

db = spi1_enrichment_terms.loc["Spi1_plot","database"]
terms = spi1_enrichment_terms.loc["Spi1_plot","terms"].split(" | ")

for direction in ['up','down']:
    
    # genetic perturbation effect in untreated and Listeria 6h
    KO = "mSpi1"
    for cond in conditions:
        if cond=="LO28_2h":
            continue

        # load enr results
        df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'DEA_condition', "{}_{}_{}".format(cond, KO, direction), db, 'Enrichr_{}_{}_{}_{}.csv'.format(db, cond, KO, direction)))

        # subset by list of terms
        df_tmp = df_tmp[df_tmp['Term'].isin(terms)]
        
        # Drop columns
        df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)
    #     df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set', 'Genes'], axis=1)

        # Rename columns 
    #     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

        # add cluster column
        df_tmp["Control"] = cond
        df_tmp["Perturbation"] = KO+" KO"
        df_tmp["Direction"] = direction
        
        # set indices
        df_tmp.set_index(["Term", "Control", "Perturbation", "Direction"], inplace=True)

        spi1_enr_dfs.append(df_tmp)

    # treatment effect in WT and Spi1 KO
    cond = "LO28_6h"
    for KO in ['mSpi1','mNon-targeting']:

        # load enr results
        df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'DEA_KO', "{}_{}_untreated_{}".format(KO, cond, direction), db, 'Enrichr_{}_{}_{}_untreated_{}.csv'.format(db, KO, cond, direction)))

        # subset by list of terms
        df_tmp = df_tmp[df_tmp['Term'].isin(terms)]
        
        # Drop columns
        df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)
    #     df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set', 'Genes'], axis=1)

        # Rename columns 
    #     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

        # add cluster column
        df_tmp["Control"] = KO+" KO"
        df_tmp["Perturbation"] = cond
        df_tmp["Direction"] = direction
        
        # set indices
        df_tmp.set_index(["Term", "Control", "Perturbation", "Direction"], inplace=True)

        spi1_enr_dfs.append(df_tmp)
        
        
        
spi1_enr_df = pd.concat(spi1_enr_dfs, axis=0)

# add database column
spi1_enr_df["Database"] = db

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = spi1_enr_df,
                     sheet_name = "Spi1_Listeria_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(spi1_enr_df.shape)
spi1_enr_df.head()

(163, 6)


,,,,Overlap,P-value,Adjusted P-value,Odds Ratio,Genes,Database
Term,Control,Perturbation,Direction,,,,,,
Antigen processing and presentation,Untreated,Spi1 KO,up,3/36,7.618237e-04,1.318541e-02,17.341912,HSP90AB1;CTSL;HSPA8,BioPlanet_2019
Arachidonic acid metabolism,Untreated,Spi1 KO,up,2/25,6.877169e-03,8.144015e-02,16.419710,GPX4;PTGDS,BioPlanet_2019
Cytoplasmic ribosomal proteins,Untreated,Spi1 KO,up,15/98,8.518237e-19,9.583016e-17,38.508564,RPS24;RPL28;RPL23;RPL32;RPS10;RPL36;RPL21;RPS1...,BioPlanet_2019
Inhibitor of DNA binding (ID) signaling pathway,Untreated,Spi1 KO,up,1/12,5.845127e-02,2.378717e-01,16.875000,ID3,BioPlanet_2019
Innate immune system,Untreated,Spi1 KO,up,2/185,2.367215e-01,3.301595e-01,2.193811,HSP90AB1;CTSL,BioPlanet_2019


In [44]:
# DEA - UT: Csf1r vs NT DEA results
dea_dfs = []

cond = "untreated"

for assay in ['RNA', 'AB']:
    
    # load dea results
    df_tmp = pd.read_csv(os.path.join(data_path, 'DEA_condition', 'KO15_DEA_{}_{}_mCsf1r.csv'.format(assay, cond)), index_col=0)

    # set index name
    df_tmp.index.names = ["Gene"]
    
    # add column
    df_tmp["Condition"]=cond
    df_tmp["Assay"]=assay
    
    # set indices
    df_tmp.reset_index(inplace=True)
    df_tmp.set_index(["Gene", "Condition", "Assay"], inplace=True)
    
    dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "DEA_Csf1r_in_Untreated",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(1115, 5)


,,,P-value,Avg log2FC,Percentage expression 1,Percentage expression 2,Adjusted P-value
Gene,Time point,Assay,,,,,
Csf1r,Untreated,RNA,2.355798e-23,-1.029912,0.383,0.821,3.342170e-19
Ei24,Untreated,RNA,5.717037e-05,0.301344,0.506,0.317,8.110760e-01
Nudt21,Untreated,RNA,1.745362e-04,0.288053,0.851,0.718,1.000000e+00
Lta4h,Untreated,RNA,1.947871e-04,0.256561,0.487,0.305,1.000000e+00
Mkrn1,Untreated,RNA,3.136294e-04,0.328329,0.708,0.565,1.000000e+00


In [45]:
# Enrichment results of NT/WT with treatment
NT_enr_dfs = []

db = "GO_Biological_Process_2021"
KO = "mNon-targeting"

for direction in ['up','down']:
    
    # treatment effect in NT/WT
    for cond in conditions:
        if cond=="untreated":
            continue

        # load enr results
        df_tmp = pd.read_csv(os.path.join(data_path, 'enrichment_analysis', 'DEA_KO', "{}_{}_untreated_{}".format(KO, cond, direction), db, 'Enrichr_{}_{}_{}_untreated_{}.csv'.format(db, KO, cond, direction)))
        
        # Drop columns
        df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)
    #     df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set', 'Genes'], axis=1)

        # Rename columns 
    #     df_tmp.columns = ["cluster_{}_{}".format(cluster,col) for col in df_tmp.columns]

        # add cluster column
        df_tmp["Time point"] = cond
        df_tmp["Direction"] = direction
        
        # set indices
        df_tmp.set_index(["Term", "Time point", "Direction"], inplace=True)

        NT_enr_dfs.append(df_tmp)

NT_enr_df = pd.concat(NT_enr_dfs, axis=0)

# add database column
NT_enr_df["Database"] = db

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = NT_enr_df,
                     sheet_name = "WT_Listeria_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(NT_enr_df.shape)
NT_enr_df.head()

(8987, 6)


,,,Overlap,P-value,Adjusted P-value,Odds Ratio,Genes,Database
Term,Time point,Direction,,,,,,
'de novo' posttranslational protein folding (GO:0051084),Listeria 2h,up,3/28,0.008184,0.037269,7.258544,HSPA5;HSPA8;DNAJB1,GO_Biological_Process_2021
3'-UTR-mediated mRNA destabilization (GO:0061158),Listeria 2h,up,4/14,0.000043,0.000828,19.468407,ZFP36L1;ZC3H12A;RC3H1;ZFP36,GO_Biological_Process_2021
3'-UTR-mediated mRNA stabilization (GO:0070935),Listeria 2h,up,2/11,0.011185,0.045893,12.273593,MAPKAPK2;ZFP36,GO_Biological_Process_2021
ATF6-mediated unfolded protein response (GO:0036500),Listeria 2h,up,1/9,0.126758,0.210114,7.466035,HSPA5,GO_Biological_Process_2021
ATP-dependent chromatin remodeling (GO:0043044),Listeria 2h,up,1/39,0.444551,0.485050,1.719285,ACTB,GO_Biological_Process_2021


In [46]:
# DEA post-mixscape KO effects within conditions
dea_dfs = []

for assay in ['RNA', 'AB']:
    for cond in conditions:
        KOs = list(metadata.loc[(metadata["mixscape_class.global"]=="KO") & (metadata["Condition"]==cond), "KO_call"].unique())
        for KO in KOs:
            
            # load dea results
            df_tmp = pd.read_csv(os.path.join(data_path, 'DEA_condition_mixscape', 'KO15_DEA_{}_{}_m{}.csv'.format(assay, cond, KO)), index_col=0)

            # set index name
            df_tmp.index.names = ["Gene"]

            # add column
            df_tmp["Time point"]=cond
            df_tmp["KO"]=KO
            df_tmp["Assay"]=assay

            # set indices
            df_tmp.reset_index(inplace=True)
            df_tmp.set_index(["Gene", "Time point","KO", "Assay"], inplace=True)

            dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "DEA_KO_effect_in_time_points",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(44298, 5)


,,,,P-value,Avg log2FC,Percentage expression 1,Percentage expression 2,Adjusted P-value
Gene,Time point,KO,Assay,,,,,
Ifit3,Untreated,Stat1,RNA,1.022991e-29,1.348961,0.736,0.160,1.370297e-25
Cxcl10,Untreated,Stat1,RNA,1.977560e-27,1.818780,0.925,0.527,2.648942e-23
Rsad2,Untreated,Stat1,RNA,8.796826e-24,1.345955,0.849,0.382,1.178335e-19
Ifit1,Untreated,Stat1,RNA,6.827681e-23,1.197628,0.717,0.210,9.145678e-19
Phf11b,Untreated,Stat1,RNA,1.169138e-20,1.070399,0.802,0.374,1.566060e-16


In [47]:
# enrichments of DEA results post-mixscape KO effects within conditions -> GO BP
enr_dfs = []
db = "GO_Biological_Process_2021"

for cond in conditions:
    KOs = list(metadata.loc[(metadata["mixscape_class.global"]=="KO") & (metadata["Condition"]==cond), "KO_call"].unique())
    for KO in KOs:
        for direction in ['up','down']:
            tmp_path = os.path.join(data_path, 'enrichment_analysis', 'DEA_condition_mixscape', '{}_m{}_{}'.format(cond, KO, direction), db, 'Enrichr_{}_{}_m{}_{}.csv'.format(db, cond, KO, direction))
            
            # check if exists
            if not(os.path.exists(tmp_path)):
                continue

            # load enrichment results
            df_tmp = pd.read_csv(tmp_path)
            
            # Drop columns
            df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)

            # add column
            df_tmp["Time point"]=cond
            df_tmp["KO"]=KO
            df_tmp["Direction"] = direction
            
            # set indices
            df_tmp.set_index(["Time point", "KO", "Direction", "Term"], inplace=True)

            enr_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
enr_df = pd.concat(enr_dfs, axis=0)

# add database column
enr_df["Database"] = db

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = enr_df,
                     sheet_name = "enrichments_KO_in_time_points",
                     column_width = 25,
                     index_format = index_format
                    )

print(enr_df.shape)
enr_df.head()

(28398, 6)


Overlap  \
Time point KO    Direction Term                                                         
Untreated  Stat1 up        C21-steroid hormone biosynthetic process (GO:00...     1/7   
                           C21-steroid hormone metabolic process (GO:0008207)     1/9   
                           CD4-positive, alpha-beta T cell activation (GO:...     1/3   
                           DNA catabolic process (GO:0006308)                     1/9   
                           DNA catabolic process, exonucleolytic (GO:0000738)     1/3   

                                                                                P-value  \
Time point KO    Direction Term                                                           
Untreated  Stat1 up        C21-steroid hormone biosynthetic process (GO:00...  0.028397   
                           C21-steroid hormone metabolic process (GO:0008207)  0.036364   
                           CD4-positive, alpha-beta T cell activation (GO:...  0.012268   
                           DNA catabolic process (GO:0006308)                  0.036364   
                           DNA catabolic process, exonucleolytic (GO:0000738)  0.012268   

                                                                               Adjusted P-value  \
Time point KO    Direction Term                                                                   
Untreated  Stat1 up        C21-steroid hormone biosynthetic process (GO:00...          0.083669   
                           C21-steroid hormone metabolic process (GO:0008207)          0.098361   
                           CD4-positive, alpha-beta T cell activation (GO:...          0.064605   
                           DNA catabolic process (GO:0006308)                          0.098361   
                           DNA catabolic process, exonucleolytic (GO:0000738)          0.064605   

                                                                               Odds Ratio  \
Time point KO    Direction Term                                                             
Untreated  Stat1 up        C21-steroid hormone biosynthetic process (GO:00...   35.417989   
                           C21-steroid hormone metabolic process (GO:0008207)   27.543210   
                           CD4-positive, alpha-beta T cell activation (GO:...   82.666667   
                           DNA catabolic process (GO:0006308)                   27.543210   
                           DNA catabolic process, exonucleolytic (GO:0000738)   82.666667   

                                                                               Genes  \
Time point KO    Direction Term                                                        
Untreated  Stat1 up        C21-steroid hormone biosynthetic process (GO:00...   TSPO   
                           C21-steroid hormone metabolic process (GO:0008207)   TSPO   
                           CD4-positive, alpha-beta T cell activation (GO:...  RSAD2   
                           DNA catabolic process (GO:0006308)                  ISG20   
                           DNA catabolic process, exonucleolytic (GO:0000738)  ISG20   

                                                                                                 Database  
Time point KO    Direction Term                                                                            
Untreated  Stat1 up        C21-steroid hormone biosynthetic process (GO:00...  GO_Biological_Process_2021  
                           C21-steroid hormone metabolic process (GO:0008207)  GO_Biological_Process_2021  
                           CD4-positive, alpha-beta T cell activation (GO:...  GO_Biological_Process_2021  
                           DNA catabolic process (GO:0006308)                  GO_Biological_Process_2021  
                           DNA catabolic process, exonucleolytic (GO:0000738)  GO_Biological_Process_2021

In [48]:
# Save Excel file
writer.save()

# CROP-seq large - TableS5

In [49]:
data_path = os.path.join('results', 'KO150')
conditions = ["untreated", "LO28-6h", "LO28-24h"]

In [50]:
# Create Excel writer
writer = pd.ExcelWriter(os.path.join(results_path,'S05_CROPseq_large.xlsx'), engine='xlsxwriter')

# Get the workbook and worksheet objects
workbook = writer.book
# define specific formatting
index_format = workbook.add_format(xlsx_format_dict)

In [51]:
# add an empty sheet called LEGEND (fill with manually curated data later)
pd.DataFrame().to_excel(writer, sheet_name="TableS5_legends", index=False)

In [52]:
# metadata
metadata = pd.read_csv(os.path.join(data_path, 'KO150_NORMALIZED_metadata.csv'), index_col=0)
# set condition column
metadata['Condition'] = metadata['hto_demux'].str.split("_", expand=True)[1]

# load mixscape classifications per condition
mixscape_dfs = []
for cond in conditions:
    cond_tmp = pd.read_csv(os.path.join(data_path, 'mixscape', cond, 'KO150_mixscape_annotations.csv'), index_col=0)
    mixscape_dfs.append(cond_tmp)
    
# load mixscape classification across condition (joint analysis)
mixscape_all = pd.read_csv(os.path.join(data_path, 'mixscape', 'all', 'KO150_mixscape_annotations.csv'), index_col=0).add_prefix('joint_analysis_')

# concatenate dfs
mixscape_df = pd.concat(mixscape_dfs, axis=0)
metadata = pd.concat([metadata, mixscape_df, mixscape_all], axis=1)

# Drop columns
metadata = metadata.drop(['old.ident', "orig.ident", "pass_QC", "hto_demux", "nCount_HTO", "nFeature_HTO"], axis=1)

# add index name
metadata.index.names = ["Cell barcode"]

# set indices
metadata.reset_index(inplace=True)
metadata.set_index(["Cell barcode", "Condition", "KO_call"], inplace=True)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = metadata.copy(deep=True),
                     sheet_name = "single_cell_annotations",
                     column_width = 25,
                     index_format = index_format
                    )

# reset indices for downstream steps
metadata.reset_index(inplace=True)
metadata.set_index(["Cell barcode"], inplace=True)

print(metadata.shape)
metadata.head()

(28303, 22)


,Condition,KO_call,nCount_RNA,nFeature_RNA,nCount_AB,nFeature_AB,nCount_gRNA,nFeature_gRNA,batch,percent.mt,...,nFeature_SCT,S.Score,G2M.Score,Phase,mixscape_class,mixscape_class_p_ko,mixscape_class.global,joint_analysis_mixscape_class,joint_analysis_mixscape_class_p_ko,joint_analysis_mixscape_class.global
Cell barcode,,,,,,,,,,,,,,,,,,,,,
A1_AAACCTGAGAATCTCC-1,untreated,Chd7,17884,4442,987,9,358,51,A1,1.481771,...,4442,0.628917,0.252506,S,Chd7 NP,0.000000,NP,Chd7 NP,0.000000,NP
A1_AAACCTGAGCGATGAC-1,untreated,Babam1,12849,3787,1378,9,1103,56,A1,1.299712,...,3787,0.033039,0.527634,G2M,Babam1 NP,0.000000,NP,Babam1 NP,0.000000,NP
A1_AAACCTGAGTGCCAGA-1,untreated,Mov10,14479,3851,458,10,838,72,A1,1.201740,...,3851,0.027528,0.626059,G2M,Mov10 NP,0.000000,NP,Mov10 NP,0.000000,NP
A1_AAACCTGAGTGGAGAA-1,untreated,Spi1,13765,3832,1437,9,270,117,A1,1.031602,...,3832,0.578333,-0.045930,S,Spi1 NP,0.054726,NP,Spi1 NP,0.067429,NP
A1_AAACCTGAGTTTAGGA-1,untreated,Jmjd1c,22985,5073,2638,10,267,42,A1,1.096367,...,5056,0.169754,-0.372522,S,Jmjd1c NP,0.000000,NP,Jmjd1c NP,0.000000,NP


In [53]:
# DEA post-mixscape KO effects within conditions
dea_dfs = []

for assay in ['RNA', 'AB']:
    for cond in conditions:
        KOs = list(metadata.loc[(metadata["mixscape_class.global"]=="KO") & (metadata["Condition"]==cond), "KO_call"].unique())
        for KO in KOs:
            
            # load dea results
            df_tmp = pd.read_csv(os.path.join(data_path, 'DEA_condition_mixscape', 'KO150_DEA_{}_{}_{}.csv'.format(assay, cond, KO)), index_col=0)

            # set index name
            df_tmp.index.names = ["Gene"]

            # add column
            df_tmp["Time point"]=cond
            df_tmp["KO"]=KO
            df_tmp["Assay"]=assay

            # set indices
            df_tmp.reset_index(inplace=True)
            df_tmp.set_index(["Gene", "Time point","KO", "Assay"], inplace=True)

            dea_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
dea_df = pd.concat(dea_dfs, axis=0)


# filter for genes that at least in one comparison were statistically significant
genes_sig = dea_df.index[(dea_df["p_val_adj"]<0.05)].get_level_values('Gene').unique().tolist()
dea_df = dea_df.loc[genes_sig,:]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = dea_df,
                     sheet_name = "DEA_KO_effect_in_time_points",
                     column_width = 25,
                     index_format = index_format
                    )

print(dea_df.shape)
dea_df.head()

(50848, 5)


P-value  Avg log2FC  \
Gene   Time point KO     Assay                             
S100a6 Untreated  Smc1a  RNA    1.813502e-18   -1.430115   
                  Ifnar1 RNA    8.592256e-02   -0.133587   
                  Sfpq   RNA    2.182019e-01   -0.125579   
                  Tyk2   RNA    3.080555e-01   -0.148946   
                  Spi1   RNA    7.399310e-06    0.562207   

                                Percentage expression 1  \
Gene   Time point KO     Assay                            
S100a6 Untreated  Smc1a  RNA                        1.0   
                  Ifnar1 RNA                        1.0   
                  Sfpq   RNA                        1.0   
                  Tyk2   RNA                        1.0   
                  Spi1   RNA                        1.0   

                                Percentage expression 2  Adjusted P-value  
Gene   Time point KO     Assay                                             
S100a6 Untreated  Smc1a  RNA                        1.0      2.692143e-14  
                  Ifnar1 RNA                        1.0      1.000000e+00  
                  Sfpq   RNA                        1.0      1.000000e+00  
                  Tyk2   RNA                        1.0      1.000000e+00  
                  Spi1   RNA                        1.0      1.098428e-01

In [54]:
# enrichments of DEA results post-mixscape KO effects within conditions
enr_dfs = []
db = "GO_Biological_Process_2021"

for cond in conditions:
    KOs = list(metadata.loc[(metadata["mixscape_class.global"]=="KO") & (metadata["Condition"]==cond), "KO_call"].unique())
    
    for KO in KOs:
        for direction in ['up','down']:
            tmp_path = os.path.join(data_path, 'enrichment_analysis', 'DEA_condition_mixscape', '{}_{}_{}'.format(cond, KO, direction), db, 'Enrichr_{}_{}_{}_{}.csv'.format(db, cond, KO, direction))
            
            # check if exists
            if not(os.path.exists(tmp_path)):
                continue

            # load enrichment results
            df_tmp = pd.read_csv(tmp_path)
            
            # Drop columns
            df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)

            # add column
            df_tmp["Time point"]=cond
            df_tmp["KO"]=KO
            df_tmp["Direction"] = direction
            
            # set indices
            df_tmp.set_index(["Time point", "KO", "Direction", "Term"], inplace=True)

            enr_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
enr_df = pd.concat(enr_dfs, axis=0)

# add database column
enr_df["Database"] = db

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = enr_df,
                     sheet_name = "enrichments_KO_in_time_points",
                     column_width = 25,
                     index_format = index_format
                    )

print(enr_df.shape)
enr_df.head()

(47160, 6)


Overlap  \
Time point KO    Direction Term                                                         
Untreated  Smc1a up        BMP signaling pathway (GO:0030509)                    1/39   
                           ERAD pathway (GO:0036503)                             1/77   
                           SCF-dependent proteasomal ubiquitin-dependent p...    1/83   
                           apoptotic process (GO:0006915)                       1/169   
                           carbohydrate derivative catabolic process (GO:1...    1/13   

                                                                                P-value  \
Time point KO    Direction Term                                                           
Untreated  Smc1a up        BMP signaling pathway (GO:0030509)                  0.036174   
                           ERAD pathway (GO:0036503)                           0.070249   
                           SCF-dependent proteasomal ubiquitin-dependent p...  0.075525   
                           apoptotic process (GO:0006915)                      0.148167   
                           carbohydrate derivative catabolic process (GO:1...  0.012196   

                                                                               Adjusted P-value  \
Time point KO    Direction Term                                                                   
Untreated  Smc1a up        BMP signaling pathway (GO:0030509)                          0.076640   
                           ERAD pathway (GO:0036503)                                   0.104537   
                           SCF-dependent proteasomal ubiquitin-dependent p...          0.109773   
                           apoptotic process (GO:0006915)                              0.173093   
                           carbohydrate derivative catabolic process (GO:1...          0.046886   

                                                                               Odds Ratio  \
Time point KO    Direction Term                                                             
Untreated  Smc1a up        BMP signaling pathway (GO:0030509)                   29.203156   
                           ERAD pathway (GO:0036503)                            14.753247   
                           SCF-dependent proteasomal ubiquitin-dependent p...   13.681186   
                           apoptotic process (GO:0006915)                        6.680018   
                           carbohydrate derivative catabolic process (GO:1...   87.763314   

                                                                               Genes  \
Time point KO    Direction Term                                                        
Untreated  Smc1a up        BMP signaling pathway (GO:0030509)                  PDCD4   
                           ERAD pathway (GO:0036503)                           FBXO2   
                           SCF-dependent proteasomal ubiquitin-dependent p...  FBXO2   
                           apoptotic process (GO:0006915)                      PDCD4   
                           carbohydrate derivative catabolic process (GO:1...  FBXO2   

                                                                                                 Database  
Time point KO    Direction Term                                                                            
Untreated  Smc1a up        BMP signaling pathway (GO:0030509)                  GO_Biological_Process_2021  
                           ERAD pathway (GO:0036503)                           GO_Biological_Process_2021  
                           SCF-dependent proteasomal ubiquitin-dependent p...  GO_Biological_Process_2021  
                           apoptotic process (GO:0006915)                      GO_Biological_Process_2021  
                           carbohydrate derivative catabolic process (GO:1...  GO_Biological_Process_2021

In [55]:
# Cross-prediction graph as adjacency matrix for each condition

for cond in conditions:
    tmp_path = os.path.join(data_path, 'KO_classifier', cond, 'KO150_clfLR_connprobNorm.csv')

    df_tmp = pd.read_csv(tmp_path, index_col=0)
            
    print(cond + " " + str(df_tmp.shape))
    
    df_tmp.index.names = [cond]

    # add dataframe as sheet to excel file
    writer = df_to_excel(writer = writer,
                         df = df_tmp,
                         sheet_name = "crossprediction_{}".format(reduce(lambda s, kv: s.replace(*kv), replacements.items(), cond).replace(" ", "_")),
                         column_width = 25,
                         index_format = index_format
                        )


untreated (21, 21)
LO28-6h (25, 25)
LO28-24h (12, 12)


In [56]:
# STRING analysis results for each condition

for cond in conditions:
    tmp_path = os.path.join(data_path, 'KO_classifier', cond, 'KO150_clfLR_graphprob_STRING_comparison.csv')

    df_tmp = pd.read_csv(tmp_path, index_col=0)
    
    # set column names
    df_tmp.columns = ["STRING interaction score", "Average crossprediction probability", "Overlap", "Unique in crossprediction", "Unique in STRING"]
            
    print(cond + " " + str(df_tmp.shape))
    
    df_tmp.index.names = ["Edge"]

    # add dataframe as sheet to excel file
    writer = df_to_excel(writer = writer,
                         df = df_tmp,
                         sheet_name = "STRING_{}".format(reduce(lambda s, kv: s.replace(*kv), replacements.items(), cond).replace(" ", "_")),
                         column_width = 25,
                         index_format = index_format
                        )

untreated (118, 5)
LO28-6h (164, 5)
LO28-24h (44, 5)


In [57]:
df_tmp.head()

,STRING interaction score,Average crossprediction probability,Overlap,Unique in crossprediction,Unique in STRING
Edge,,,,,
Stat2-Tyk2,0.994,0.106390,True,False,False
Spi1-Tyk2,0.205,0.000000,False,False,True
Spi1-Stat2,0.376,0.000000,False,False,True
Smc1a-Spi1,0.000,0.162550,False,True,False
Sfpq-Spi1,0.000,0.052689,False,True,False


In [58]:
# # Save Excel file
# writer.save()

# Joint perturbation analysis & Integrative Analysis - included in TableS5

In [62]:
# Cross-prediction graph as adjacency matrix across all conditions

cond = "all"
tmp_path = os.path.join(data_path, 'KO_classifier', cond, 'KO150_clfLR_connprobNorm.csv')

df_tmp = pd.read_csv(tmp_path, index_col=0)

df_tmp.index.names = ["Joint analysis"]

print(cond + " " + str(df_tmp.shape))

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = df_tmp,
                     sheet_name = "joint_crossprediction_analysis",
                     column_width = 25,
                     index_format = index_format
                    )

print(df_tmp.shape)
df_tmp.head()

all (58, 58)
(58, 58)


,Listeria 24h+Chd4 KO,Listeria 24h+Ep400 KO,Listeria 24h+Ifnar1 KO,Listeria 24h+Ikzf1 KO,Listeria 24h+Irf9 KO,Listeria 24h+Jak1 KO,Listeria 24h+Myd88 KO,Listeria 24h+Sfpq KO,Listeria 24h+Smc1a KO,Listeria 24h+Spi1 KO,...,Untreated+Jak1 KO,Untreated+Med14 KO,Untreated+Runx1 KO,Untreated+Sf3b1 KO,Untreated+Sfpq KO,Untreated+Smc1a KO,Untreated+Spi1 KO,Untreated+Stat2 KO,Untreated+Tyk2 KO,Untreated+Yeats2 KO
Joint analysis,,,,,,,,,,,,,,,,,,,,,
Listeria 24h+Chd4 KO,0.000000,0.008974,0.007204,0.025959,0.016927,0.010049,0.021910,0.049981,0.042560,0.013292,...,0.000871,0.008110,0.008865,0.001710,0.003715,0.003435,0.002087,0.001014,0.004084,0.003158
Listeria 24h+Ep400 KO,0.008861,0.000000,0.016035,0.055534,0.013130,0.047038,0.037502,0.006059,0.025426,0.015297,...,0.001112,0.001540,0.003305,0.006563,0.003341,0.001363,0.003347,0.002193,0.002919,0.014476
Listeria 24h+Ifnar1 KO,0.001781,0.002817,0.000000,0.004260,0.460557,0.197412,0.003267,0.004518,0.002764,0.002279,...,0.012862,0.000647,0.000117,0.001258,0.000107,0.000536,0.000099,0.010740,0.002304,0.000278
Listeria 24h+Ikzf1 KO,0.017429,0.017485,0.005919,0.000000,0.037531,0.015704,0.070946,0.015897,0.031763,0.022469,...,0.001733,0.004647,0.002164,0.002642,0.004280,0.004644,0.002851,0.001185,0.002078,0.005183
Listeria 24h+Irf9 KO,0.002619,0.003358,0.385940,0.011316,0.000000,0.164542,0.006078,0.002579,0.003927,0.000792,...,0.031174,0.000387,0.000328,0.001361,0.000063,0.000488,0.000179,0.013295,0.004183,0.000829


In [63]:
# Cross-prediction graph scores: Within-knockout and within-condition

cond = "all"
tmp_path = os.path.join(data_path, 'KO_classifier', cond, 'KO150_clfLR_graphprob_scores.csv')

df_tmp = pd.read_csv(tmp_path, index_col=0)

df_tmp.index.names = ["KO gene"]

# rename columns
df_tmp.columns = ["KO_similarity", "Time point_similarity"]

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = df_tmp,
                     sheet_name = "joint_crossprediction_scores",
                     column_width = 25,
                     index_format = index_format
                    )

print(df_tmp.shape)
df_tmp.head()

(20, 2)


,KO similarity,Time point similarity
KO gene,,
Arid1a,0.415096,0.538811
Brd2,0.469509,0.534615
Chd4,0.453651,0.446404
Dnmt1,0.411223,0.538753
Dnttip2,0.214832,0.566572


In [64]:
# Enrichment analysis results of overlapping DEGs of edges in crossprediction graph
enr_dfs = []
db = "BioPlanet_2019"

for cond1 in conditions:
    for cond2 in conditions:
        for KO1 in metadata['KO_call'].unique():
            for KO2 in metadata['KO_call'].unique():
                for direction in ['up', 'down']:
                    edge_desc = '{}+{}-{}+{}_{}'.format(cond1, KO1, cond2, KO2, direction)
                    tmp_path = os.path.join(data_path, 'KO_classifier', 'all', 'interpretation', 'enrichment_analysis', 'stat', edge_desc, db, 'Enrichr_{}_{}.csv'.format(db, edge_desc))

                    # check if exists
                    if not(os.path.exists(tmp_path)):
                        continue
                    
                    # load enrichment results
                    df_tmp = pd.read_csv(tmp_path)
            
                    # Drop columns
                    df_tmp = df_tmp.drop(['Unnamed: 0', 'Gene_set'], axis=1)

                    # add column
                    df_tmp["KO 1"] = KO1
                    df_tmp["Time point 1"] = cond1
                    df_tmp["KO 2"] = KO2
                    df_tmp["Time point 2"] = cond2
                    df_tmp["Direction"] = direction
            
                    # set indices
                    df_tmp.set_index(["Time point 1", "KO 1", "Time point 2", "KO 2", "Direction", "Term"], inplace=True)

                    enr_dfs.append(df_tmp)

# dea_df = pd.concat(dea_dfs, axis=1)
enr_df = pd.concat(enr_dfs, axis=0)

# add database column
enr_df["Database"] = db

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = enr_df,
                     sheet_name = "joint_graph_edge_enrichments",
                     column_width = 25,
                     index_format = index_format
                    )

print(enr_df.shape)
enr_df.head()

(9657, 6)


Overlap  \
Time point 1 KO 1   Time point 2 KO 2 Direction Term                                                         
Untreated    Ifnar1 Untreated    Tyk2 down      Activation of IRF3/IRF7 mediated by TBK1/IKK ep...    1/15   
                                                Antiviral mechanism by interferon-stimulated genes    1/64   
                                                Apoptosis                                            1/204   
                                                Carbohydrate metabolism                              1/165   
                                                Cell adhesion molecules (CAMs)                        1/57   

                                                                                                     P-value  \
Time point 1 KO 1   Time point 2 KO 2 Direction Term                                                           
Untreated    Ifnar1 Untreated    Tyk2 down      Activation of IRF3/IRF7 mediated by TBK1/IKK ep...  0.008057   
                                                Antiviral mechanism by interferon-stimulated genes  0.033982   
                                                Apoptosis                                           0.104815   
                                                Carbohydrate metabolism                             0.085555   
                                                Cell adhesion molecules (CAMs)                      0.030315   

                                                                                                    Adjusted P-value  \
Time point 1 KO 1   Time point 2 KO 2 Direction Term                                                                   
Untreated    Ifnar1 Untreated    Tyk2 down      Activation of IRF3/IRF7 mediated by TBK1/IKK ep...          0.019135   
                                                Antiviral mechanism by interferon-stimulated genes          0.046118   
                                                Apoptosis                                                   0.107648   
                                                Carbohydrate metabolism                                     0.090308   
                                                Cell adhesion molecules (CAMs)                              0.042665   

                                                                                                    Odds Ratio  \
Time point 1 KO 1   Time point 2 KO 2 Direction Term                                                             
Untreated    Ifnar1 Untreated    Tyk2 down      Activation of IRF3/IRF7 mediated by TBK1/IKK ep...  141.238095   
                                                Antiviral mechanism by interferon-stimulated genes   32.993304   
                                                Apoptosis                                            10.252801   
                                                Carbohydrate metabolism                              12.709957   
                                                Cell adhesion molecules (CAMs)                       37.062657   

                                                                                                    Genes  \
Time point 1 KO 1   Time point 2 KO 2 Direction Term                                                        
Untreated    Ifnar1 Untreated    Tyk2 down      Activation of IRF3/IRF7 mediated by TBK1/IKK ep...   IRF7   
                                                Antiviral mechanism by interferon-stimulated genes  ISG15   
                                                Apoptosis                                            IRF7   
                                                Carbohydrate metabolism                              SDC3   
                                                Cell adhesion molecules (CAMs)                       SDC3   

                                                                                                          Database  
Time point 1 KO 1   Ti

In [65]:
# Enrichment analysis results of KO perturbation signatures in temporal deviating gene clusters
cluster_to_letter = {"LO28_cluster_1": 'A', "LO28_cluster_2": 'B', "LO28_cluster_3": 'C', "LO28_cluster_4": 'D', "LO28_cluster_5": 'E', "LO28_cluster_6": 'F', "LO28_cluster_7": 'G'}
listeria_changes = {
    'G': 'A',
    'A': 'B',
    'B': 'C',
    'D': 'D',
    'E': 'E',
    'C': 'F',
    'F': 'G',
}

tmp_path = os.path.join(data_path, 'special_analysis', 'INT_comparison', 'GSEA_split_results.csv')

df_tmp = pd.read_csv(tmp_path)

# remove all but Listeria clusters
df_tmp = df_tmp[df_tmp['Term'].str.contains('LO28')]

# adapt cluster labels to letters
df_tmp["Term"] = df_tmp["Term"].replace(cluster_to_letter)
df_tmp["Term"] = df_tmp["Term"].replace(listeria_changes)

# split columns
df_tmp[['Time point', 'KO', 'Direction']] = df_tmp['Gene_set'].str.split("_", expand=True)

# Drop columns
df_tmp = df_tmp.drop(['Unnamed: 0', 'Odds Ratio1', 'Gene_set'], axis=1)

# rename column 
df_tmp.rename(columns={'Term': 'Listeria_Cluster'}, inplace=True)

# set indices
df_tmp.set_index(['Time point', 'KO', 'Direction'], inplace=True)

# add dataframe as sheet to excel file
writer = df_to_excel(writer = writer,
                     df = df_tmp,
                     sheet_name = "KO_effects_enriched_in_Listeria",
                     column_width = 25,
                     index_format = index_format
                    )

print(df_tmp.shape)
df_tmp.head()

(288, 6)


Listeria Cluster  Overlap   P-value  \
Time point KO   Direction                                       
Untreated  Spi1 up                       B    4/474  0.143547   
                up                       C    2/244  0.278980   
                up                       F    1/802  0.970034   
                up                       D  10/1260  0.040454   
                up                       E    2/553  0.687566   

                           Adjusted P-value  Odds Ratio  \
Time point KO   Direction                                 
Untreated  Spi1 up                 0.358868    2.273010   
                up                 0.467925    2.415019   
                up                 0.970034    0.420999   
                up                 0.199170    2.082048   
                up                 0.747354    1.047277   

                                                                       Genes  
Time point KO   Direction                                                     
Untreated  Spi1 up                            S100A4;TMEM176A;TMEM176B;PDE4B  
                up                                               CTNNB1;CCL5  
                up                                                     RPL21  
                up         GSTM1;H2-Q6;H2-T22;H2-Q4;CTSL;FBXO32;LY6E;MGST...  
                up                                               KANK3;PNRC1

In [66]:
# Save Excel file
writer.save()